In [1]:
# Data handling and processing
import pandas as pd
import numpy as np
import random
import os 
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# Model evaluation and utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore', message="is_sparse is deprecated")
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import roc_auc_score, brier_score_loss

# XGBoost
import xgboost as xgb
print(xgb.__version__)

# Statsmodels
import statsmodels.api as sm

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import torchmetrics
from torch.nn import functional as F
from torch.nn import CrossEntropyLoss

# Optuna for hyperparameter optimization
import optuna
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import log_loss
import numpy as np

from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

2.1.1


### Prepare NN input

In [2]:
def process_sample(sample):
    # Define the paths based on the sample name
    qb_path = f'/data2/1000Genome/{sample}/qb/{sample}'
    readcount_path = f'/data2/1000Genome/{sample}/beastie/runModel_phased_even100/chr1-22_alignBiasp0.05_s0.7_a0.05_sinCov0_totCov1_W1000K1000/tmp/{sample}'
    
    # Check if the 'beastie' folder exists (we already filtered for this, so this is optional now)
    if not os.path.isdir(f'/data2/1000Genome/{sample}/beastie'):
        return

    # Read the qb file and extract qb_label
    qb_data = pd.read_csv(f"{qb_path}_qb_highestsite.tsv", sep='\t')
    qb_data["qb_label"] = qb_data["qb_mode"]
    qb_label = qb_data[["geneID", "qb_label"]]

    # Read the input readcount file
    input_readcount = pd.read_csv(f"{readcount_path}_real_alignBiasafterFilter.phasedByshapeit2.cleaned.tsv", sep='\t')
    input_readcount = input_readcount[["geneID", "chrN", "pos", "refCount", "altCount"]]

    # Read the genetic features file
    genetic_features = pd.read_csv(f"{readcount_path}.meta.w_error.tsv", sep='\t')
    genetic_features = genetic_features[["geneID", "pos", "MAF", "min_MAF", "diff_MAF", "d", "r2", "log10_distance"]]

    # Perform inner join of the genetic features and the input readcount
    input_df = pd.merge(input_readcount, genetic_features, how="inner", on=["geneID", "pos"])
    input_df["individual"] = sample

    # Read the ancestry file and get the first word
    ancestry_file = f'/data2/1000Genome/{sample}/ancestry'
    with open(ancestry_file, 'r') as file:
        ancestry = file.readline().strip()

    input_df["ancestry"] = ancestry

    # Format the geneID column
    input_df["geneID"] = input_df["geneID"].str.split(".").str[0]

    # Merge input_df with qb_label
    input_df = pd.merge(input_df, qb_label, how="inner", on=["geneID"])

    # Save the final DataFrame as a TSV file
    output_file = f"{qb_path}_NN_input.tsv"
    input_df.to_csv(output_file, sep='\t', index=False)


In [16]:
# Find only the samples with the 'beastie' directory
base_dir = '/data2/1000Genome/'
samples = [
    sample for sample in os.listdir(base_dir)
    if os.path.isdir(os.path.join(base_dir, sample)) and
    os.path.isdir(f'/data2/1000Genome/{sample}/beastie')
]

# Process each sample with a progress bar
for sample in tqdm(samples, desc="Processing samples"):
    process_sample(sample)

Processing samples: 100%|██████████| 445/445 [01:07<00:00,  6.58it/s]


### Randomly select some individuals for a training set and testing set

In [17]:
def select_and_split_samples(ancestry_group, num_individuals_train=10, num_individuals_test=10, base_dir='/data2/1000Genome/'):
    # Collect output file paths for all individuals in the specified ancestry group
    ancestry_files = []
    for sample in os.listdir(base_dir):
        sample_dir = os.path.join(base_dir, sample)
        ancestry_file = os.path.join(sample_dir, 'ancestry')
        output_file = os.path.join(sample_dir, 'qb', f"{sample}_NN_input.tsv")
        
        # Check if both ancestry file and output file exist
        if os.path.isfile(ancestry_file) and os.path.isfile(output_file):
            with open(ancestry_file, 'r') as file:
                ancestry = file.readline().strip()
            # Collect the file path if it matches the specified ancestry group
            if ancestry == ancestry_group:
                ancestry_files.append(output_file)
    
    # Ensure we have enough individuals to split into train and test sets
    total_required = num_individuals_train + num_individuals_test
    if len(ancestry_files) < total_required:
        raise ValueError(f"Not enough individuals in {ancestry_group} ancestry group. Required: {total_required}, Found: {len(ancestry_files)}")
    
    # Randomly select the specified number of unique individuals for training and testing
    selected_files = random.sample(ancestry_files, total_required)
    train_files = selected_files[:num_individuals_train]
    test_files = selected_files[num_individuals_train:num_individuals_train + num_individuals_test]
    
    # Combine data for the training set
    train_df = pd.DataFrame()
    for file_path in train_files:
        df = pd.read_csv(file_path, sep='\t')
        train_df = pd.concat([train_df, df], ignore_index=True)
    
    # Combine data for the test set
    test_df = pd.DataFrame()
    for file_path in test_files:
        df = pd.read_csv(file_path, sep='\t')
        test_df = pd.concat([test_df, df], ignore_index=True)
    
    return train_df, test_df


In [18]:
# Example usage
train_df, test_df = select_and_split_samples('GBR')

In [25]:
train_df.head()

,geneID,chrN,pos,refCount,altCount,MAF,min_MAF,diff_MAF,d,r2,log10_distance,individual,ancestry,qb_label
0,ENSG00000177757,1,753541,3,0,0.1233,NaN,NaN,NaN,NaN,NaN,HG00148,GBR,0.419205
1,ENSG00000225880,1,761732,1,1,0.1302,0.1233,0.0069,NaN,NaN,NaN,HG00148,GBR,0.522619
2,ENSG00000225880,1,761752,1,0,0.1292,0.1292,0.0010,1.0,1.000,1.301030,HG00148,GBR,0.522619
3,ENSG00000225880,1,761881,1,2,0.0427,0.0427,0.0865,1.0,0.005,2.110590,HG00148,GBR,0.522619
4,ENSG00000225880,1,761958,1,2,0.0497,0.0427,0.0070,1.0,1.000,1.886491,HG00148,GBR,0.522619


### modify the testset

In [32]:
def prepare_data_for_cnn_individual(data, label_col='qb_label'):
    # Step 1: Determine the maximum number of hets per gene across individuals
    max_hets_per_gene = data.groupby(['individual', 'geneID']).size().max()
    
    # Step 2: Calculate relative position for each gene by individual
    # Sort by geneID and position for proper ordering
    data = data.sort_values(by=['individual', 'geneID', 'pos'])
    data['relative_pos'] = data.groupby(['individual', 'geneID'])['pos'].diff().fillna(0)
    
    # Step 3: Filter relevant columns - dropping geneID, chrN, and pos
    feature_columns = ['refCount', 'altCount', 'MAF', 'relative_pos']
    X_data = []
    y_data = []
    
    # Group by individual and geneID to form CNN inputs
    grouped = data.groupby(['individual', 'geneID'])
    
    for (individual, geneID), group in grouped:
        # Extract the feature columns for the current individual and gene
        X = group[feature_columns].to_numpy()
        
        # Pad the feature array to have the same number of rows
        padded_X = np.zeros((max_hets_per_gene, len(feature_columns)))
        padded_X[:X.shape[0], :] = X  # Fill in the available data
        
        # Get the label (assumes it's the same across all rows in the group)
        y = group[label_col].iloc[0]
        
        # Append to the lists
        X_data.append(padded_X)
        y_data.append(y)
    
    # Convert the lists to numpy arrays suitable for CNN input
    X_data = np.array(X_data)
    y_data = np.array(y_data)
    
    return X_data, y_data

In [36]:
data = train_df
X, y = prepare_data_for_cnn_individual(data)
print("X shape:", X.shape)
print("y shape:", y.shape)


In [19]:
train_df.head()

,geneID,chrN,pos,refCount,altCount,MAF,min_MAF,diff_MAF,d,r2,log10_distance,individual,ancestry,qb_label
0,ENSG00000177757,1,753541,3,0,0.1233,NaN,NaN,NaN,NaN,NaN,HG00148,GBR,0.419205
1,ENSG00000225880,1,761732,1,1,0.1302,0.1233,0.0069,NaN,NaN,NaN,HG00148,GBR,0.522619
2,ENSG00000225880,1,761752,1,0,0.1292,0.1292,0.0010,1.0,1.000,1.301030,HG00148,GBR,0.522619
3,ENSG00000225880,1,761881,1,2,0.0427,0.0427,0.0865,1.0,0.005,2.110590,HG00148,GBR,0.522619
4,ENSG00000225880,1,761958,1,2,0.0497,0.0427,0.0070,1.0,1.000,1.886491,HG00148,GBR,0.522619


test_df.head()